## Introduccion CrewAI

Importamos las dependencias 


In [ ]:
from crewai import Agent, Crew, Task, LLM
import os
import json
import yaml
import groq

from dotenv import load_dotenv

load_dotenv()

ModuleNotFoundError: No module named 'crewai'

### Importamos LLM para escoger el modelo que vamos a utilizar

Aqui podemos ver todos los proveedores posibles:
https://docs.crewai.com/concepts/llms

In [ ]:
llm = LLM(model="gpt-4o-mini")

### Archivos YAML

En CrewAI, se suelen utilizar archivos yaml, para los prompts, lo que ayuda a que todo este mucho mejor organizado.

In [3]:
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Creamos las herramientas

Hay varias formas de crear herramientas en CrewAI, podemos utilizar el decorador @tool, como vimos en la clase anterior, o tambien podemos hacerlo con la clase BaseTool.

https://docs.crewai.com/concepts/tools

CrewAI, nos proporciona una gran cantidad de herramientas bastante potentes ya construidas para leer archivos, scrapear webs, hacer busquedas RAG...

https://docs.crewai.com/tools/


En este caso, vamos a crear una herramienta que busque el tiempo, otra que busque las actividades disponibles en Nantes, y por ultimo vamos a analizar una imagen a traves de una herramienta.

In [4]:
from crewai.tools import tool
import http.client
from datetime import datetime
import random

import http.client
import json
import os
from langchain.tools import tool

@tool
def get_weather(city: str, country_code: str = None) -> str:
    """
    Obtiene información del tiempo para una ciudad específica.
    
    Args:
        city: Nombre de la ciudad
        country_code: Código del país en formato ISO (opcional, ej: ES, FR)
    
    Returns:
        str: Información del tiempo formateada
    """
    try:
        conn = http.client.HTTPSConnection("open-weather13.p.rapidapi.com")
        
        # Verificar que las credenciales estén disponibles
        api_key = os.getenv("X_RAPIDAPI_KEY")
        api_host = os.getenv("X_RAPIDAPI_HOST")

        
        if not api_key or not api_host:
            return "Error: No se encontraron las credenciales de API en las variables de entorno"
        
        headers = {
            'X-RapidAPI-Key': api_key,
            'X-RapidAPI-Host': api_host
        }
        
        # Construir la ruta correctamente
        if country_code:
            path = f"/city/{city}/{country_code}"
        else:
            path = f"/city/{city}"
            
        print(f"Requesting: {path}")  # Para debugging
        
        conn.request("GET", path, headers=headers)
        response = conn.getresponse()
        
        if response.status != 200:
            return f"Error: La API respondió con código {response.status}"
            
        data = json.loads(response.read().decode("utf-8"))
        
        # Verificar que los datos necesarios estén presentes
        if 'main' not in data or 'weather' not in data:
            return "Error: Datos del clima incompletos en la respuesta"
            
        weather_info = (
            f"Tiempo en {city}:\n"
            f"Temperatura: {data['main']['temp']}°C\n"
            f"Sensación térmica: {data['main']['feels_like']}°C\n"
            f"Humedad: {data['main']['humidity']}%\n"
            f"Descripción: {data['weather'][0]['description']}"
        )
        
        return weather_info
        
    except json.JSONDecodeError:
        return "Error: No se pudo decodificar la respuesta JSON"
    except Exception as e:
        return f"Error al obtener el tiempo: {str(e)}"
    finally:
        conn.close()



@tool
def get_nantes_activities() -> str:
    """
    Consulta las actividades disponibles hoy en Nantes.
    
    Returns:
        str: Lista de actividades disponibles en Nantes para hoy
    """
    # Lista de actividades predefinidas
    activities = [
        {
            "nombre": "Exposición de Arte Contemporáneo",
            "lugar": "Château des Ducs de Bretagne",
            "horario": "10:00 - 18:00",
            "precio": "12€"
        },
        {
            "nombre": "Concierto de Jazz",
            "lugar": "Le Lieu Unique",
            "horario": "20:30 - 23:00",
            "precio": "15€"
        },
        {
            "nombre": "Tour en Les Machines de l'île",
            "lugar": "Île de Nantes",
            "horario": "10:00 - 19:00",
            "precio": "8.50€"
        },
        {
            "nombre": "Mercado Local Gastronómico",
            "lugar": "Place Talensac",
            "horario": "8:00 - 13:00",
            "precio": "Entrada libre"
        },
        {
            "nombre": "Taller de Vinos del Loira",
            "lugar": "La Cave du Loire",
            "horario": "16:00 - 18:00",
            "precio": "25€"
        }
    ]
    
    # Seleccionar aleatoriamente 2-3 actividades para "hoy"
    today_activities = random.sample(activities, random.randint(2, 3))
    
    # Formatear la respuesta
    response = f"Actividades disponibles hoy en Nantes ({datetime.now().strftime('%d/%m/%Y')}):\n\n"
    
    for activity in today_activities:
        response += (
            f"🎯 {activity['nombre']}\n"
            f"📍 Lugar: {activity['lugar']}\n"
            f"🕒 Horario: {activity['horario']}\n"
            f"💶 Precio: {activity['precio']}\n\n"
        )
    
    return response



import base64
from openai import OpenAI
from typing import Optional

@tool
def analyze_image(image_path: str, prompt: Optional[str] = "What is in this image?") -> str:
    """
    Analiza una imagen utilizando el modelo GPT-4 Vision de OpenAI.
    
    Args:
        image_path: Ruta al archivo de imagen a analizar
        prompt: Pregunta o prompt específico para analizar la imagen (opcional)
    
    Returns:
        str: Descripción o análisis de la imagen
    """
    try:
        # Inicializar el cliente de OpenAI
        client = OpenAI()
        
        # Codificar la imagen en base64
        def encode_image(image_path):
            with open(image_path, "rb") as image_file:
                return base64.b64encode(image_file.read()).decode("utf-8")
        
        # Obtener la imagen en formato base64
        base64_image = encode_image(image_path)
        
        # Crear la solicitud a la API
        response = client.chat.completions.create(
            model="gpt-4o",  # Asegurarse de usar el modelo correcto
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt,
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            },
                        },
                    ],
                }
            ],
            max_tokens=300  # Ajustar según necesidades
        )
        
        # Extraer y retornar la respuesta
        return response.choices[0].message.content
        
    except Exception as e:
        return f"Error al analizar la imagen: {str(e)}"

In [5]:
get_weather.run({
    'city':'Nantes',
    'country_code': 'NTE'
})

Requesting: /city/Nantes/NTE


'Tiempo en Nantes:\nTemperatura: 49.93°C\nSensación térmica: 49.21°C\nHumedad: 87%\nDescripción: overcast clouds'

## Construimos el sistema de Agentes

Iniciamos los agentes, las tareas y les pasamos la configuracion.

Por ultimo inciamos la crew, en este caso, va a seguir un flujo secuencial, pero podriamos modificarlo para que fuese jerarquico.



In [6]:
# Creating Agents
extractor_tareas = Agent(
  config=agents_config['extractor_tareas'],
  llm=llm,
  tools=[analyze_image]
)

organizador = Agent(
  config=agents_config['organizador'],
  llm=llm
)

freetime_planner = Agent(
  config=agents_config['freetime_planner'],
  tools=[get_weather, get_nantes_activities],
  llm=llm
)

organizador_final = Agent(
  config=agents_config['organizador_final'],
  llm=llm
)

# Creating Tasks
extraer_tareas = Task(
  config=tasks_config['extraer_tareas'],
  agent=extractor_tareas
)

organizar_tareas = Task(
  config=tasks_config['organizar_tareas'],
  agent=organizador,
  
)

planear_freetime = Task(
  config=tasks_config['planear_freetime'],
  agent=freetime_planner,

)

crear_horario = Task(
  config=tasks_config['crear_horario'],
  agent=organizador_final,
  context=[organizar_tareas, planear_freetime]
)

# Creating Crew
crew = Crew(
  agents=[
    extractor_tareas,
    organizador,
    freetime_planner,
    organizador_final
  ],
  tasks=[
    extraer_tareas,
    organizar_tareas,
    planear_freetime,
    crear_horario
  ],
  verbose=True
)

In [ ]:
url = "temp_images/IMG_3852.jpeg"
result = crew.kickoff(inputs={'image': url})

# Agent: Task List Analysis Specialist
## Task: - Analyze provided images containing handwritten or digital task lists - Extract and convert task items into structured text format - Perform complexity analysis for each task:
  * Evaluate cognitive load requirements (focus, mental effort, expertise needed)
  * Assess urgency level based on implicit and explicit time indicators
- Apply consistent evaluation criteria across all tasks - Consider context clues in the image that might affect task interpretation - Handle both clear and ambiguous task descriptions - Account for different handwriting styles and digital formats - Maintain accuracy in text extraction while preserving original meaning - DO NOT create subtask just organize the ones you have received. temp_images/IMG_3852.jpeg



# Agent: Task List Analysis Specialist
## Using tool: analyze_image
## Tool Input: 
"{\"image_path\": \"temp_images/IMG_3852.jpeg\", \"prompt\": null}"
## Tool Output: 
Error al analizar la imagen: Error co

In [9]:
from IPython.display import Markdown

markdown  = result.raw
Markdown(markdown)

NameError: name 'result' is not defined

## Medimos el coste de la iteracion

In [8]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0016


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,10836,7318,1024,3518,7


In [ ]:
crew.plot()